# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aykhal,66.0000,111.5000,-21.43,100,100,2.36,RU,1707169269
1,1,meadow lakes,61.6247,-149.6011,0.35,85,0,0.00,US,1707169270
2,2,west island,-12.1568,96.8225,26.99,89,100,3.09,CC,1707169270
3,3,crane,31.3974,-102.3501,15.11,36,4,2.11,US,1707169270
4,4,nemuro,43.3236,145.5750,-7.71,78,98,5.79,JP,1707169270


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size= "Humidity",
    color = "City",
    alpha = 0.5
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
narrow_data_df = city_data_df.loc[(city_data_df["Max Temp"]<29)&(city_data_df["Max Temp"]>21)&
                                       (city_data_df["Wind Speed"]<3.5)&(city_data_df["Cloudiness"]==0),:]

# Drop any rows with null values
ideal_city_data_df= narrow_data_df.dropna()

# Display sample data
ideal_city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,port saint john's,-31.6229,29.5448,23.32,92,0,2.67,ZA,1707169290
202,202,karwar,14.8000,74.1333,24.68,64,0,1.68,IN,1707169316
281,281,east london,-33.0153,27.9116,23.44,86,0,2.36,ZA,1707169342
283,283,kindia,10.0569,-12.8658,22.37,49,0,2.05,GN,1707169343
362,362,pamandzi,-12.7967,45.2794,26.88,94,0,1.54,YT,1707169369
414,414,salalah,17.0151,54.0924,23.05,60,0,1.54,OM,1707169379
431,431,tekkali,18.6167,84.2333,21.21,92,0,1.88,IN,1707169383
441,441,kampli,15.4000,76.6167,23.26,54,0,3.17,IN,1707169385
482,482,saint-pierre,-21.3393,55.4781,21.82,73,0,1.54,RE,1707169394
527,527,taltal,-25.4000,-70.4833,23.32,73,0,2.15,CL,1707169403


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
68,port saint john's,ZA,-31.6229,29.5448,92,
202,karwar,IN,14.8000,74.1333,64,
281,east london,ZA,-33.0153,27.9116,86,
283,kindia,GN,10.0569,-12.8658,49,
362,pamandzi,YT,-12.7967,45.2794,94,
414,salalah,OM,17.0151,54.0924,60,
431,tekkali,IN,18.6167,84.2333,92,
441,kampli,IN,15.4000,76.6167,54,
482,saint-pierre,RE,-21.3393,55.4781,73,
527,taltal,CL,-25.4000,-70.4833,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params = params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port saint john's - nearest hotel: Outback Inn
karwar - nearest hotel: Emerald Bay, Karwar, Sterling Holidays
east london - nearest hotel: No hotel found
kindia - nearest hotel: Hôtel buffet de la gare de Kindia
pamandzi - nearest hotel: Gite Le Madina
salalah - nearest hotel: Muscat International Hotel
tekkali - nearest hotel: No hotel found
kampli - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
taltal - nearest hotel: Hostería Taltal


,City,Country,Lat,Lng,Humidity,Hotel Name
68,port saint john's,ZA,-31.6229,29.5448,92,Outback Inn
202,karwar,IN,14.8000,74.1333,64,"Emerald Bay, Karwar, Sterling Holidays"
281,east london,ZA,-33.0153,27.9116,86,No hotel found
283,kindia,GN,10.0569,-12.8658,49,Hôtel buffet de la gare de Kindia
362,pamandzi,YT,-12.7967,45.2794,94,Gite Le Madina
414,salalah,OM,17.0151,54.0924,60,Muscat International Hotel
431,tekkali,IN,18.6167,84.2333,92,No hotel found
441,kampli,IN,15.4000,76.6167,54,No hotel found
482,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
527,taltal,CL,-25.4000,-70.4833,73,Hostería Taltal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size= "Humidity",
    color = "City",
    hover_cols=["Country","Hotel Name"]
)
# Display the map
# YOUR CODE HERE
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)